In [1]:
from IPython.display import display, HTML
import pandas as pd
import sqlite3
from sqlite3 import Error

def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(create_table_sql, conn):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows

In [2]:
conn = create_connection("normalized_ppg6.db", delete_db=True)

In [3]:
import os
def loadFiles(foldername='Datasets', fileformat='.csv'):
    csvfiles = list(filter(lambda f: f.endswith(f'{fileformat}'), os.listdir(f'./{foldername}')))
    csvfiles = [os.path.abspath(f"{foldername}/{file}") for file in csvfiles]
    return(csvfiles)

csvfiles = loadFiles()
csvfiles

['C:\\Users\\iamh2k\\H2K\\PyhtonProject\\Datasets\\AAPL_5Y.csv',
 'C:\\Users\\iamh2k\\H2K\\PyhtonProject\\Datasets\\AMZN_5Y.csv',
 'C:\\Users\\iamh2k\\H2K\\PyhtonProject\\Datasets\\Company.csv',
 'C:\\Users\\iamh2k\\H2K\\PyhtonProject\\Datasets\\FB_5Y.csv',
 'C:\\Users\\iamh2k\\H2K\\PyhtonProject\\Datasets\\GOOG_5Y.csv',
 'C:\\Users\\iamh2k\\H2K\\PyhtonProject\\Datasets\\NFLX_5Y.csv',
 'C:\\Users\\iamh2k\\H2K\\PyhtonProject\\Datasets\\TSLA_5Y.csv']

In [4]:
def loadDatabase(file_list=csvfiles, connx=conn, overwrite=True,):
    for filepath in csvfiles:
        df = pd.read_csv(filepath)
        filename = os.path.basename(filepath)
        table_name = filename.rsplit('.',1)[0]
        write_mode = 'replace' if overwrite else 'append'
        df.to_sql(table_name, connx, if_exists=write_mode, index=False)

loadDatabase()

C:\Users\iamh2k\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\core\generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [5]:
sql_statement = "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;"
testdf = execute_sql_statement(sql_statement, conn)
display(testdf)

[('AAPL_5Y',),
 ('AMZN_5Y',),
 ('Company',),
 ('FB_5Y',),
 ('GOOG_5Y',),
 ('NFLX_5Y',),
 ('TSLA_5Y',)]

In [6]:
sql_statement = "SELECT 'AAPL' AS StockSymbol, * FROM AAPL_5Y LIMIT 10;"
testdf = execute_sql_statement(sql_statement, conn)
display(testdf)

[('AAPL',
  '2016-12-12',
  113.290001,
  115.0,
  112.489998,
  113.300003,
  110.569832,
  26374377),
 ('AAPL',
  '2016-12-13',
  113.839996,
  115.919998,
  113.75,
  115.190002,
  112.414291,
  43733811),
 ('AAPL',
  '2016-12-14',
  115.040001,
  116.199997,
  114.980003,
  115.190002,
  112.414291,
  34031834),
 ('AAPL',
  '2016-12-15',
  115.379997,
  116.730003,
  115.230003,
  115.82,
  113.029099,
  46524544),
 ('AAPL',
  '2016-12-16',
  116.470001,
  116.5,
  115.644997,
  115.970001,
  113.175507,
  44351134),
 ('AAPL',
  '2016-12-19',
  115.800003,
  117.379997,
  115.75,
  116.639999,
  113.829353,
  27779423),
 ('AAPL',
  '2016-12-20',
  116.739998,
  117.5,
  116.68,
  116.949997,
  114.131874,
  21424965),
 ('AAPL',
  '2016-12-21',
  116.800003,
  117.400002,
  116.779999,
  117.059998,
  114.239235,
  23724084),
 ('AAPL',
  '2016-12-22',
  116.349998,
  116.510002,
  115.639999,
  116.290001,
  113.487785,
  26085854),
 ('AAPL',
  '2016-12-23',
  115.589996,
  116.5199

In [7]:
sql_statement = """CREATE TABLE HistoricalPrices AS
SELECT * FROM
(SELECT 'AAPL' AS StockSymbol, * FROM AAPL_5Y
UNION ALL
SELECT 'AMZN' AS StockSymbol, * FROM AMZN_5Y
UNION ALL
SELECT 'FB' AS StockSymbol, * FROM FB_5Y
UNION ALL
SELECT 'GOOG' AS StockSymbol, * FROM GOOG_5Y
UNION ALL
SELECT 'NFLX' AS StockSymbol, * FROM NFLX_5Y);"""
create_table(sql_statement, conn)

In [8]:
sql_statement = "SELECT * FROM HistoricalPrices LIMIT 1;"
testdf = execute_sql_statement(sql_statement, conn)
display(testdf)

[('AAPL',
  '2016-12-12',
  113.290001,
  115.0,
  112.489998,
  113.300003,
  110.569832,
  26374377)]

In [9]:
sql_statement = "SELECT * FROM Company LIMIT 1;"
testdf = execute_sql_statement(sql_statement, conn)
display(testdf)

[(1,
  'Apple Inc.',
  'AAPL',
  '2.944T',
  '1980-12-12',
  '$22.00',
  '$101.2M',
  '$1.2B')]